In [1]:
import math
import torch
import gpytorch
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from LBFGS import FullBatchLBFGS
torch.set_default_tensor_type(torch.DoubleTensor)
import gc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
#data=pd.read_csv('QingData_dbp_19f.csv')
data = pd.read_csv('data_47p_1h_21f.csv')
#data=data.sample(frac=0.5,random_state=1)
patient_list=data['patient_id'].unique()
#print(patient_list)
####################################SBP 加‘20’
#feature=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19']
feature=['HrRR','HrSS','AmFS','AmSN','AmFN','AmFN_FS','AmFN_SN','TmFN','TmNF','TmFN_NF','Tm_FS','Tm_SF','Tm_SN','Tm_FQ','PAT_S','PAT_F','PAT_Q','ArFS','ArSN','ArNF','ArNF_FN']
n_devices = 3#torch.cuda.device_count()
print('Planning to run on {} GPUs.'.format(n_devices))

Planning to run on 3 GPUs.


In [3]:
# L=len(patient_list)
# for id in range(L):
#     target_data=data[data['patient_id']==patient_list[id]]
#     print(id,patient_list[id])
#     target_data['SBP'].plot.hist(grid=True, bins=50,rwidth=0.9,color='#607c8e')
#     plt.show()

In [ ]:
MAE=0
ME=0
STD=0
L=len(patient_list)
error_vector=[]
for id in range(L):
    source_data=data[data['patient_id']!=patient_list[id]]
    target_data=data[data['patient_id']==patient_list[id]]
    
    train_x=source_data[feature]
    test_x=target_data[feature]
    scaler=StandardScaler()
    train_x=scaler.fit_transform(train_x)
    test_x=scaler.transform(test_x)
    
    train_x=torch.from_numpy( np.array(train_x) )
    train_y=torch.from_numpy( np.array(source_data['SBP']) )
    test_x=torch.from_numpy( np.array(test_x) )
    test_y=torch.from_numpy( np.array(target_data['SBP']))
    
    output_device = torch.device('cuda:0')
    train_x, train_y = train_x.to(output_device), train_y.to(output_device)
    test_x, test_y = test_x.to(output_device), test_y.to(output_device)
    
    class ExactGPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood, n_devices):
            super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            base_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5))

            self.covar_module = gpytorch.kernels.MultiDeviceKernel(
                base_covar_module, device_ids=range(n_devices),
                output_device=output_device
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def train(train_x,
              train_y,
              n_devices,
              output_device,
              checkpoint_size,
              preconditioner_size,
              n_training_iter,
    ):
        likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
        model = ExactGPModel(train_x, train_y, likelihood, n_devices).to(output_device)
        model.train()
        likelihood.train()

        optimizer = FullBatchLBFGS(model.parameters(), lr=0.1)
        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


        with gpytorch.beta_features.checkpoint_kernel(checkpoint_size), \
             gpytorch.settings.max_preconditioner_size(preconditioner_size):

            def closure():
                optimizer.zero_grad()
                output = model(train_x)
                loss = -mll(output, train_y)
                return loss

            loss = closure()
            loss.backward()

            for i in range(n_training_iter):
                options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
                loss, _, _, _, _, _, _, fail = optimizer.step(options)

                print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                    i + 1, n_training_iter, loss.item(),
                    model.covar_module.module.base_kernel.lengthscale.item(),
                    model.likelihood.noise.item()
                ))

                if fail:
                    print('Convergence reached!')
                    break

        print(f"Finished training on {train_x.size(0)} data points using {n_devices} GPUs.")
        return model, likelihood
    
    def find_best_gpu_setting(train_x,
                              train_y,
                              n_devices,
                              output_device,
                              preconditioner_size
    ):
        N = train_x.size(0)

        # Find the optimum partition/checkpoint size by decreasing in powers of 2
        # Start with no partitioning (size = 0)
        settings = [0] + [int(n) for n in np.ceil(N / 2**np.arange(1, np.floor(np.log2(N))))]

        for checkpoint_size in settings:
            print('Number of devices: {} -- Kernel partition size: {}'.format(n_devices, checkpoint_size))
            try:
                # Try a full forward and backward pass with this setting to check memory usage
                _, _ = train(train_x, train_y,
                             n_devices=n_devices, output_device=output_device,
                             checkpoint_size=checkpoint_size,
                             preconditioner_size=preconditioner_size, n_training_iter=1)

                # when successful, break out of for-loop and jump to finally block
                break
            except RuntimeError as e:
                print('RuntimeError: {}'.format(e))
            except AttributeError as e:
                print('AttributeError: {}'.format(e))
            finally:
                # handle CUDA OOM error
                gc.collect()
                torch.cuda.empty_cache()
        return checkpoint_size

    # Set a large enough preconditioner size to reduce the number of CG iterations run
    preconditioner_size = 100
    checkpoint_size = find_best_gpu_setting(train_x, train_y,
                                            n_devices=n_devices,
                                            output_device=output_device,
                                            preconditioner_size=preconditioner_size)
    
    
    model, likelihood = train(train_x, train_y,
                              n_devices=n_devices, output_device=output_device,
                              checkpoint_size=checkpoint_size,
                              preconditioner_size=preconditioner_size,
                              n_training_iter=100)
    
    model.eval()
    likelihood.eval()
    with torch.no_grad(),\
         gpytorch.settings.fast_pred_var(),\
         gpytorch.beta_features.checkpoint_kernel(checkpoint_size):        
        y_preds = likelihood(model(test_x))
        y_mean=y_preds.mean
    
    
    mae=mean_absolute_error(test_y.cpu(),y_mean.cpu())
    MAE+=mae
    error=(test_y.cpu()-y_mean.cpu()).numpy()
    error_vector=np.append(error_vector,error)
    me=np.mean(error)
    ME+=me
    std=np.std(error,ddof=1)
    STD+=std*(test_x.cpu().shape[0]-1)
    print(patient_list[id],'MAE: ', format(mae,'.3f'),'ME',format(me,'.3f'),'STD: ',format(std,'.3f'))


prediction_error={'SBP': np.array(data['SBP']),
                 'prediction error': error_vector}
prediction_error=pd.DataFrame(prediction_error)
prediction_error.to_csv('SBP GPR.csv')###
plt.title("SBP of GPR.csv") ###
plt.xlabel("true value of SBP") #
plt.ylabel("prediction error") 
plt.scatter(np.array(data['SBP']),error_vector.transpose(),s=0.2) #  
plt.show()
print('average MAE: ', format(MAE/L,'.3f'),'ME: ', format(ME/L,'.3f'), 'STD: ', format(STD/(data.shape[0]-L),'.3f'))

Number of devices: 3 -- Kernel partition size: 0
RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/kernel.py", line 398, in __call__
    res = lazify(super(Kernel, self).__call__(x1_, x2_, last_dim_is_batch=last_dim_is_batch, **params))
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/module.py", line 28, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/scale_kernel.py", line 92, in forward
    orig_output = self.base_kernel.forward(x1, x2, diag=diag, last_dim_is_batch=last_dim_is_batch, **params)
  File "/home/staff_qing_liu/.co

RuntimeError: CUDA out of memory. Tried to allocate 1.22 GiB (GPU 1; 10.92 GiB total capacity; 8.62 GiB already allocated; 363.38 MiB free; 9.83 GiB reserved in total by PyTorch)
Number of devices: 3 -- Kernel partition size: 1959


/home/staff_qing_liu/Jiacheng /LBFGS.py:257: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554788289/work/torch/csrc/utils/python_arg_parser.cpp:1005.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


Iter 1/1 - Loss: 12.890   lengthscale: 39.797   noise: 3.541
Finished training on 125321 data points using 3 GPUs.
Iter 1/100 - Loss: 12.890   lengthscale: 39.797   noise: 3.541
Iter 2/100 - Loss: 12.174   lengthscale: 39.787   noise: 3.882
Iter 3/100 - Loss: 11.412   lengthscale: 39.770   noise: 4.309
Iter 4/100 - Loss: 10.737   lengthscale: 39.744   noise: 4.759
Iter 5/100 - Loss: 10.117   lengthscale: 39.708   noise: 5.247
Iter 6/100 - Loss: 9.551   lengthscale: 39.660   noise: 5.771
Iter 7/100 - Loss: 9.029   lengthscale: 39.596   noise: 6.339
Iter 8/100 - Loss: 8.548   lengthscale: 39.513   noise: 6.953
Iter 9/100 - Loss: 8.103   lengthscale: 39.408   noise: 7.618
Iter 10/100 - Loss: 7.687   lengthscale: 39.276   noise: 8.340
Iter 11/100 - Loss: 7.304   lengthscale: 39.113   noise: 9.117
Iter 12/100 - Loss: 6.949   lengthscale: 38.898   noise: 9.958
Iter 13/100 - Loss: 6.621   lengthscale: 38.619   noise: 10.861
Iter 14/100 - Loss: 6.316   lengthscale: 38.257   noise: 11.834
Iter 

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/kernel.py", line 398, in __call__
    res = lazify(super(Kernel, self).__call__(x1_, x2_, last_dim_is_batch=last_dim_is_batch, **params))
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/module.py", line 28, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/scale_kernel.py", line 101, in forward
    return orig_output.mul(outputscales)
RuntimeError: CUDA out of memory. Tried to allocate 4.87 GiB (GPU 0; 10.92 GiB total capacity; 5.35 GiB already allocated; 4.48 GiB free; 5.65 GiB r

...

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/kernel.py", line 398, in __call__
    res = lazify(super(Kernel, self).__call__(x1_, x2_, last_dim_is_batch=last_dim_is_batch, **params))
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/module.py", line 28, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/scale_kernel.py", line 92, in forward
    orig_output = self.base_kernel.forward(x1, x2, diag=diag, last_dim_is_batch=last_dim_is_batch, **params)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpy

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/kernel.py", line 398, in __call__
    res = lazify(super(Kernel, self).__call__(x1_, x2_, last_dim_is_batch=last_dim_is_batch, **params))
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/module.py", line 28, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/scale_kernel.py", line 92, in forward
    orig_output = self.base_kernel.forward(x1, x2, diag=diag, last_dim_is_batch=last_dim_is_batch, **params)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpy

Iter 14/100 - Loss: 6.254   lengthscale: 38.448   noise: 11.685
Iter 15/100 - Loss: 5.978   lengthscale: 37.986   noise: 12.713
Iter 16/100 - Loss: 5.724   lengthscale: 37.398   noise: 13.813
Iter 17/100 - Loss: 5.487   lengthscale: 36.649   noise: 14.991
Iter 18/100 - Loss: 5.272   lengthscale: 35.706   noise: 16.241
Iter 19/100 - Loss: 5.069   lengthscale: 34.489   noise: 17.587
Iter 20/100 - Loss: 4.881   lengthscale: 32.939   noise: 19.011
Iter 21/100 - Loss: 4.701   lengthscale: 30.919   noise: 20.541
Iter 22/100 - Loss: 4.527   lengthscale: 28.240   noise: 22.191
Iter 23/100 - Loss: 4.348   lengthscale: 24.523   noise: 24.017
Iter 24/100 - Loss: 4.132   lengthscale: 18.730   noise: 26.259
Iter 25/100 - Loss: 3.568   lengthscale: 3.308   noise: 31.174
Iter 26/100 - Loss: 3.542   lengthscale: 3.138   noise: 31.219
Iter 27/100 - Loss: 3.541   lengthscale: 3.107   noise: 31.224
Iter 28/100 - Loss: 3.536   lengthscale: 3.057   noise: 31.233
Iter 29/100 - Loss: 3.556   lengthscale: 3.0

Number of devices: 3 -- Kernel partition size: 7828
RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/kernel.py", line 398, in __call__
    res = lazify(super(Kernel, self).__call__(x1_, x2_, last_dim_is_batch=last_dim_is_batch, **params))
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/module.py", line 28, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/scale_kernel.py", line 92, in forward
    orig_output = self.base_kernel.forward(x1, x2, diag=diag, last_dim_is_batch=last_dim_is_batch, **params)
  File "/home/staff_qing_liu/

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/kernel.py", line 398, in __call__
    res = lazify(super(Kernel, self).__call__(x1_, x2_, last_dim_is_batch=last_dim_is_batch, **params))
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/module.py", line 28, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpytorch/kernels/scale_kernel.py", line 92, in forward
    orig_output = self.base_kernel.forward(x1, x2, diag=diag, last_dim_is_batch=last_dim_is_batch, **params)
  File "/home/staff_qing_liu/.conda/envs/jiacheng/lib/python3.8/site-packages/gpy